In [1]:
from transformers import AutoModelForCausalLM

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from peft import PeftModel, PeftConfig
import torch

config = PeftConfig.from_pretrained("/mnt/workspace/qwen")
model = AutoModelForCausalLM.from_pretrained("/mnt/workspace/Qwen2.5-7B-Instruct", trust_remote_code=True, torch_dtype=torch.half,  device_map="auto")
lora_model = PeftModel.from_pretrained(model, "/mnt/workspace/qwen")

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]


In [3]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("/mnt/workspace/Qwen2.5-7B-Instruct")

In [4]:
import pandas as pd
data=pd.read_json("/mnt/workspace/20250208181531_camp_data_step_1_without_answer.jsonl",lines=True)
examples=pd.read_json("/mnt/workspace/20250208183027_example_output.jsonl",lines=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
out=pd.DataFrame({'id':[],'bingli_json':[],'feature_content':[]})
for i in range(0,3000):
    prompt = "以下是患者的病历信息，请你根据提供的信息生成\"诊断\"和\"诊断依据\"，\"诊断\"即患者所患病症的诊断，\"诊断依据\"即从病历信息中提取的做出诊断的依据。以下是信息：\n"
    prompt+=data.iloc[i]["feature_content"]
    messages = [
        {"role": "system", "content": "你是一个专业医生"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = lora_model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
 
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    t={'id':data.iloc[i]["id"],'bingli_json':response,'feature_content':data.iloc[i]["feature_content"]}
    out.loc[i]=t
    print(f"{i}结束")

out.to_json("output.jsonl",lines=True,orient="records")


0结束
1结束
2结束
3结束
4结束
5结束
6结束
7结束
8结束
9结束
10结束
11结束
12结束
13结束
14结束
15结束
16结束
17结束
18结束
19结束
20结束
21结束
22结束
23结束
24结束
25结束
26结束
27结束
28结束
29结束
30结束
31结束
32结束
33结束
34结束
35结束
36结束
37结束
38结束
39结束
40结束
41结束
42结束
43结束
44结束
45结束
46结束
47结束
48结束
49结束
50结束
51结束
52结束
53结束
54结束
55结束
56结束
57结束
